# FPCM & source localization: demonstration

May 2025

Daria Kleeva 

dkleeva@gmail.com

## Import

In [ ]:
import mne
import os
import matplotlib.pyplot as plt
from source_loc import fit_spike_dipoles, plot_modeled_topos, plot_dipoles_2d
from utils import compute_performance
from fpcm_detector import detect_spikes_fpcm
from summary import make_epochs, plot_average_joint, plot_spike_topographies, overlay_spline_fit_grid

In [ ]:
from pathlib import Path

SUBJECT='Simulated patient'
path = Path("/mnt/y/vzuev/datasets/eeg_etc/FPCMdata/sim_raw.fif")
path.exists()

In [ ]:
raw = mne.io.read_raw_fif(path)
raw.load_data()
raw.filter(1,40)
raw.pick('grad')
raw.crop(0,100) #for quick demo
true_peaks = [float(f"{i}.2") for i in range(0, 100)]

## Detection

In [ ]:
results = detect_spikes_fpcm(
    raw,
    peak_hw_ms = 30, # Half-width of the sharp spike (in milliseconds)
    wave_hw_ms = 90, # Half-width of the slow wave following the spike (in milliseconds)
    bkg_coeff   = 3, # Signal-to-noise ratio
    err_peak_th = 0.3, # Maximum allowed relative fitting error for the spike segment
    err_wave_th = 0.9, # Maximum allowed relative fitting error for the wave segment
    hit_threshold = 3, # Minimum number of channels that must agree on a spike to accept it
) 

In [ ]:
metrics = compute_performance(
    true_latencies_s = true_peaks,  
    results          = results,                
    raw              = raw,                    
    tolerance_ms     = 40,                     
)

In [ ]:
epochs = make_epochs(raw, results, tmin=-0.5, tmax=0.5)
plot_average_joint(epochs, title='Average spike')
plt.show()

In [ ]:
# Each subplot shows the spatial distribution of MEG signal at spike peak.   
# Yellow dots indicate the "hit" channels that contributed to detection.

# Show only first 30 spikes for the fast rendering
num_show=30
results_short=results.copy()
for key in ['peaks_samples', 'events']:
    results_short[key]=results_short[key][:num_show]

plot_spike_topographies(raw, results_short, n_cols=7) #Replace results_short with results for the full set
plt.show()

In [ ]:
#Each subplot shows the raw data (black) and the fitted spline (red dashed) for one spike and one channel.
overlay_spline_fit_grid(raw, results_short, n_cols=7, unit='uV',
                                data_kw=dict(color='0.3'),
                                spline_kw=dict(color='r', ls='--'))
#Replace results_short with results for the full set
plt.show()

## Dipole fitting

In [ ]:
# Create the cortical source space, compute BEM model and build the forward model
subjects_dir = path.parent / "fsaverage"
subjects_dir.mkdir(exist_ok=True)
mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir)

subject      = "fsaverage"
src = mne.setup_source_space(
    subject, spacing="ico4",  
    add_dist=False, subjects_dir=subjects_dir, verbose=False)
model = mne.make_bem_model(subject=subject, subjects_dir=subjects_dir )
bem = mne.make_bem_solution(model)
trans = "fsaverage"
fwd = mne.make_forward_solution(epochs.info, trans=trans, src=src, bem=bem, 
                                eeg=False, meg=epochs.get_channel_types(picks='data', unique=True)[0], n_jobs=3)

In [ ]:
# Apply RAP-MUSIC algorithm to each spike using the forward model.  

fit_res = fit_spike_dipoles(
    epochs, # Spike-centered epochs (each epoch = one spike)
    fwd, # Forward model (leadfield matrix)
    t_window=(-0.1, 0.1),  # Time window (in seconds) around the spike apex used for source fitting
    thr_music=0.8, # Threshold for subspace correlation (only dipoles with corr > thr_music are accepted)
    thr_svd=0.95,  # Variance threshold for the signal subspace (used in SVD for RAP-MUSIC)
)

In [ ]:
# All localized dipoles are projected onto the fsaverage cortical surface.  
plot_dipoles_2d(
        fit_res, 
        trans=subject,       
        subject=subject,
        subjects_dir=os.getenv("SUBJECTS_DIR"),
        color="crimson",
)
plt.show()

In [ ]:
# For each spike, compare the measured topography at the spike peak  
# with the synthetic topography generated from the fitted dipoles.  

# Show first 30 for fast rendering
show_num=30
fit_res_short=fit_res.copy()
for key in fit_res_short.keys():
    fit_res_short[key]=fit_res_short[key][:show_num]

plot_modeled_topos(epochs[:show_num], fit_res_short, fwd,
                              n_cols=4, cmap="RdBu_r", vmax=None)
plt.show()